In [1]:
!pip install transformers torch sentencepiece tqdm pandas openpyxl --quiet

In [4]:
import pandas as pd
from google.colab import files
uploaded = files.upload() #upload dataset

#load cleaned dataset
df = pd.read_excel("news_dataset_cleaned.xlsx")
df.head()

Saving news_dataset_cleaned.xlsx to news_dataset_cleaned (1).xlsx


,id,text,label,source,normalized_text,clean_text
0,1,Produk Alkohol Tersenarai dalam Barangan Bantu...,fake,sebenarnya.my,Produk Alkohol Tersenarai dalam Barangan Bantu...,produk alkohol tersenarai dalam barangan bantu...
1,2,KTM Berhad - National Day Gift Through the que...,fake,sebenarnya.my,KTM Berhad - National Day Gift Through the que...,ktm berhad national day gift through the quest...
2,3,Malaysian state threatens to jail Muslim men w...,real,The Guardian,Malaysian state threatens to jail Muslim men w...,malaysian state threatens to jail muslim men w...
3,4,Suicide will be officially decriminalised star...,real,Reddit,Suicide will be officially decriminalised star...,suicide will be officially decriminalised star...
4,5,"Law repealed, suicide attempts no longer crim...",real,The Sun,"Law repealed, suicide attempts no longer criminal",law repealed suicide attempts no longer criminal


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

#load model
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#sentiment analysis function
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        label = torch.argmax(probs, dim=1).item()
    return ["negative", "neutral", "positive"][label]

#apply sentiment analysis
tqdm.pandas()
df["sentiment"] = df["clean_text"].progress_apply(get_sentiment)
df.head()


100%|██████████| 300/300 [00:03<00:00, 82.91it/s] 


,id,text,label,source,normalized_text,clean_text,sentiment
0,1,Produk Alkohol Tersenarai dalam Barangan Bantu...,fake,sebenarnya.my,Produk Alkohol Tersenarai dalam Barangan Bantu...,produk alkohol tersenarai dalam barangan bantu...,neutral
1,2,KTM Berhad - National Day Gift Through the que...,fake,sebenarnya.my,KTM Berhad - National Day Gift Through the que...,ktm berhad national day gift through the quest...,neutral
2,3,Malaysian state threatens to jail Muslim men w...,real,The Guardian,Malaysian state threatens to jail Muslim men w...,malaysian state threatens to jail muslim men w...,negative
3,4,Suicide will be officially decriminalised star...,real,Reddit,Suicide will be officially decriminalised star...,suicide will be officially decriminalised star...,neutral
4,5,"Law repealed, suicide attempts no longer crim...",real,The Sun,"Law repealed, suicide attempts no longer criminal",law repealed suicide attempts no longer criminal,negative


In [6]:
df.to_excel("news_dataset_sentiment.xlsx", index=False)

from google.colab import files
files.download("news_dataset_sentiment.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>